# Process Data for GNN

In [1]:
import pandas as pd

/opt/homebrew/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# data from yahoo finace
usd_twd = pd.read_csv('macro/usd_twd_rate.csv')
vix = pd.read_csv('macro/vix.csv')
crude_oil = pd.read_csv('macro/crude_oil.csv')
gold = pd.read_csv('macro/gold.csv')

# data from FRED
cpi = pd.read_csv('macro/cpi.csv')
unemployment_rate = pd.read_csv('macro/unemployment_rate.csv')
interest_rate = pd.read_csv('macro/interest_rate.csv')
m2 = pd.read_csv('macro/m2_data.csv')

print(usd_twd)
print(cpi)

            Date       Open       High        Low      Close  Volume  \
0     2004-03-24  32.022999  32.022999  32.022999  32.022999       0   
1     2004-03-25  32.002998  32.002998  32.002998  32.002998       0   
2     2004-03-26  32.011002  32.011002  32.011002  32.011002       0   
3     2004-03-29  31.978001  31.978001  31.978001  31.978001       0   
4     2004-03-30  31.952000  31.952000  31.952000  31.952000       0   
...          ...        ...        ...        ...        ...     ...   
4860  2024-07-22  32.780998  32.903000  32.762901  32.780998       0   
4861  2024-07-23  32.808498  32.849998  32.766800  32.808498       0   
4862  2024-07-24  32.751301  32.799000  32.669300  32.751301       0   
4863  2024-07-25  32.714802  32.790001  32.613800  32.714802       0   
4864  2024-07-26  32.715000  32.862999  32.641998  32.800999       0   

      Dividends  Stock Splits  
0           0.0           0.0  
1           0.0           0.0  
2           0.0           0.0  
3      

## Only Keep Close price

In [3]:
usd_twd = usd_twd[['Date', 'Close']].rename(columns={'Close': 'USD/TWD'})
vix = vix[['Date', 'Close']].rename(columns={'Close': 'VIX'})
crude_oil = crude_oil[['Date', 'Close']].rename(columns={'Close': ' Crude Oil'})
gold = gold[['Date', 'Close']].rename(columns={'Close': 'Gold'})

print(usd_twd)

            Date    USD/TWD
0     2004-03-24  32.022999
1     2004-03-25  32.002998
2     2004-03-26  32.011002
3     2004-03-29  31.978001
4     2004-03-30  31.952000
...          ...        ...
4860  2024-07-22  32.780998
4861  2024-07-23  32.808498
4862  2024-07-24  32.751301
4863  2024-07-25  32.714802
4864  2024-07-26  32.800999

[4865 rows x 2 columns]


## Resample to a consistent frequency and extend to 2024-06-30

In [4]:
def resampling(df, col_name):
    # Set date as index
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # Resample to daily data
    df = df.resample('D').asfreq()
    df[col_name] = df[col_name].interpolate(method='linear')

    # Extend dates with the value from 2024-06-01
    date_range = pd.date_range(start='2024-06-02', end='2024-06-30', freq='D')
    df_value = df.loc['2024-06-01', col_name]
    extended_data = pd.DataFrame({col_name: df_value}, index=date_range)
    df = pd.concat([df, extended_data])

    # reset the index to match all data
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Date'}, inplace=True)

    return df

In [5]:
cpi = resampling(cpi, 'CPI')
unemployment_rate = resampling(unemployment_rate, 'Unemployment Rate')
interest_rate = resampling(interest_rate, 'Interest Rate')
m2 = resampling(m2, 'M2')

print(cpi)

            Date         CPI
0     1947-01-01   21.480000
1     1947-01-02   21.484516
2     1947-01-03   21.489032
3     1947-01-04   21.493548
4     1947-01-05   21.498065
...          ...         ...
28301 2024-06-26  313.049000
28302 2024-06-27  313.049000
28303 2024-06-28  313.049000
28304 2024-06-29  313.049000
28305 2024-06-30  313.049000

[28306 rows x 2 columns]


# Concat all data

In [6]:
def datetime(df):
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    return df

all_data = [usd_twd, vix, crude_oil, gold, cpi, unemployment_rate, interest_rate, m2]
all_data = [datetime(data) for data in all_data]

combined_df = pd.concat(all_data, axis=1)
combined_df.dropna(inplace=True)
print(combined_df)
combined_df.to_csv('macro/combined.csv')


              USD/TWD        VIX   Crude Oil       Gold         CPI  \
Date                                                                  
2004-03-24  32.022999  19.809999   37.009998  37.009998  187.322581   
2004-03-25  32.002998  17.879999   35.509998  35.509998  187.332258   
2004-03-26  32.011002  17.330000   35.730000  35.730000  187.341935   
2004-03-29  31.978001  16.500000   35.450001  35.450001  187.370968   
2004-03-30  31.952000  16.280001   36.250000  36.250000  187.380645   
...               ...        ...         ...        ...         ...   
2024-06-24  32.362999  13.330000   81.629997  81.629997  313.049000   
2024-06-25  32.321499  12.840000   80.830002  80.830002  313.049000   
2024-06-26  32.473499  12.550000   80.900002  80.900002  313.049000   
2024-06-27  32.582298  12.240000   81.739998  81.739998  313.049000   
2024-06-28  32.521999  12.440000   81.540001  81.540001  313.049000   

            Unemployment Rate  Interest Rate            M2  
Date           

## Normalization (60 days)

In [11]:
def normalize(df):
    rolling_min = df.rolling(window=60).min()
    rolling_max = df.rolling(window=60).max()
    normalized_df = (df-rolling_min)/(rolling_max-rolling_min)
    return normalized_df

normalized_df = normalize(combined_df)
normalized_df = normalized_df.loc['2014-01-01':'2024-06-30']
normalized_df.fillna(0.0, inplace=True)

print(normalized_df)
normalized_df.to_csv('macro/normalized_macro.csv')

             USD/TWD       VIX   Crude Oil      Gold  CPI  Unemployment Rate  \
Date                                                                           
2014-01-02  0.458912  0.275304    0.292638  0.292638  1.0           0.000000   
2014-01-03  0.936789  0.211876    0.154995  0.154995  1.0           0.006211   
2014-01-06  0.567636  0.210201    0.105509  0.105509  1.0           0.025316   
2014-01-07  1.000000  0.112829    0.135509  0.135509  1.0           0.032258   
2014-01-08  1.000000  0.105101    0.002967  0.002967  1.0           0.041096   
...              ...       ...         ...       ...  ...                ...   
2024-06-24  0.618778  0.199457    0.613470  0.613470  0.0           1.000000   
2024-06-25  0.558919  0.132972    0.554905  0.554905  0.0           1.000000   
2024-06-26  0.767356  0.093623    0.560029  0.560029  0.0           1.000000   
2024-06-27  0.931929  0.051560    0.621522  0.621522  0.0           1.000000   
2024-06-28  0.840719  0.078697    0.6068